In [2]:
import sqlite3
from tkinter import Tk, Label, Entry, Button, Listbox, Scrollbar, StringVar, END
from datetime import datetime
import os

# Database setup
def initialize_db():
    # Lấy đường dẫn gốc của thư mục làm việc hiện tại
    current_working_directory = os.getcwd()
    
    conn = sqlite3.connect(os.path.join(current_working_directory, "sales_management.db"))
    cursor = conn.cursor()
    # cursor.execute('''CREATE TABLE IF NOT EXISTS MatHang (
    #     id VARCHAR(10) PRIMARY KEY,
    #     ten_hang VARCHAR(255),
    #     don_vi VARCHAR(20),
    #     so_luong INTEGER,
    #     gia_le INTEGER,
    #     gia_si INTEGER,
    #     ngay_tao DATE DEFAULT CURRENT_DATE,
    #     is_active BOOLEAN DEFAULT 1
    # )''')
    # cursor.execute('''INSERT INTO MatHang (id, ten_hang, don_vi, so_luong, gia_le, gia_si)
    #                   VALUES ('MH001', 'Bánh mì', 'Cái', 100, 5000, 4500)''')
    # cursor.execute('''INSERT INTO MatHang (id, ten_hang, don_vi, so_luong, gia_le, gia_si)
    #                   VALUES ('MH002', 'Coca Cola', 'Lon', 50, 10000, 9000)''')
    # cursor.execute('''
    #                CREATE INDEX IF NOT EXISTS idx_id ON MatHang (id)
    #                ''')
    conn.commit()
    conn.close()

# GUI setup
class SalesManagementApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Sales Management - MatHang")

        # Input fields
        Label(root, text="ID:").grid(row=0, column=0, padx=5, pady=5)
        self.id_var = StringVar()
        Entry(root, textvariable=self.id_var).grid(row=0, column=1, padx=5, pady=5)

        Label(root, text="Tên hàng:").grid(row=1, column=0, padx=5, pady=5)
        self.ten_hang_var = StringVar()
        Entry(root, textvariable=self.ten_hang_var).grid(row=1, column=1, padx=5, pady=5)

        Label(root, text="Đơn vị:").grid(row=2, column=0, padx=5, pady=5)
        self.don_vi_var = StringVar()
        Entry(root, textvariable=self.don_vi_var).grid(row=2, column=1, padx=5, pady=5)

        Label(root, text="Số lượng:").grid(row=3, column=0, padx=5, pady=5)
        self.so_luong_var = StringVar()
        Entry(root, textvariable=self.so_luong_var).grid(row=3, column=1, padx=5, pady=5)

        Label(root, text="Giá lẻ:").grid(row=4, column=0, padx=5, pady=5)
        self.gia_le_var = StringVar()
        Entry(root, textvariable=self.gia_le_var).grid(row=4, column=1, padx=5, pady=5)

        Label(root, text="Giá sỉ:").grid(row=5, column=0, padx=5, pady=5)
        self.gia_si_var = StringVar()
        Entry(root, textvariable=self.gia_si_var).grid(row=5, column=1, padx=5, pady=5)

        # Buttons
        Button(root, text="Thêm", command=self.create_item).grid(row=6, column=0, padx=5, pady=5)
        Button(root, text="Sửa", command=self.update_item).grid(row=6, column=1, padx=5, pady=5)
        Button(root, text="Xóa", command=self.delete_item).grid(row=6, column=2, padx=5, pady=5)
        Button(root, text="Làm mới", command=self.refresh_list).grid(row=6, column=3, padx=5, pady=5)

        # Listbox
        self.listbox = Listbox(root, width=100, height=20)
        self.listbox.grid(row=7, column=0, columnspan=4, padx=5, pady=5)

        # Scrollbar
        scrollbar = Scrollbar(root, orient="vertical", command=self.listbox.yview)
        scrollbar.grid(row=7, column=4, sticky="ns")
        self.listbox.configure(yscrollcommand=scrollbar.set)

        self.refresh_list()

    # CRUD operations
    def create_item(self):
        conn = sqlite3.connect("sales_management.db")
        cursor = conn.cursor()
        ngay_tao = datetime.now().strftime("%Y-%m-%d")
        is_active = 1

        try:
            cursor.execute('''INSERT INTO MatHang (id, ten_hang, don_vi, so_luong, gia_le, gia_si, ngay_tao, is_active)
                              VALUES (?, ?, ?, ?, ?, ?, ?, ?)''',
                           (self.id_var.get(), self.ten_hang_var.get(), self.don_vi_var.get(), int(self.so_luong_var.get()),
                            int(self.gia_le_var.get()), int(self.gia_si_var.get()), ngay_tao, is_active))
            conn.commit()
            self.refresh_list()
        except Exception as e:
            print(f"Error: {e}")
        finally:
            conn.close()

    def update_item(self):
        conn = sqlite3.connect("sales_management.db")
        cursor = conn.cursor()

        try:
            cursor.execute('''UPDATE MatHang
                              SET ten_hang = ?, don_vi = ?, so_luong = ?, gia_le = ?, gia_si = ?, is_active = ?
                              WHERE id = ?''',
                           (self.ten_hang_var.get(), self.don_vi_var.get(), int(self.so_luong_var.get()),
                            int(self.gia_le_var.get()), int(self.gia_si_var.get()), 1, self.id_var.get()))
            conn.commit()
            self.refresh_list()
        except Exception as e:
            print(f"Error: {e}")
        finally:
            conn.close()

    def delete_item(self):
        conn = sqlite3.connect("sales_management.db")
        cursor = conn.cursor()

        try:
            cursor.execute("DELETE FROM MatHang WHERE id = ?", (self.id_var.get(),))
            conn.commit()
            self.refresh_list()
        except Exception as e:
            print(f"Error: {e}")
        finally:
            conn.close()

    def refresh_list(self):
        self.listbox.delete(0, END)
        conn = sqlite3.connect("sales_management.db")
        cursor = conn.cursor()

        try:
            cursor.execute("SELECT * FROM MatHang ORDER BY ten_hang")
            rows = cursor.fetchall()
            for row in rows:
                self.listbox.insert(END, row)
        except Exception as e:
            print(f"Error: {e}")
        finally:
            conn.close()

# Run application
if __name__ == "__main__":
    # initialize_db()
    root = Tk()
    app = SalesManagementApp(root)
    root.mainloop()


In [ ]:
from whoosh.index import create_in, open_dir
from whoosh.fields import Schema, TEXT, ID
from whoosh.qparser import QueryParser
import os
import shutil

# Định nghĩa schema cho mặt hàng
schema_mat_hang = Schema(
    id=ID(stored=True, unique=True),
    ten_mat_hang=TEXT(stored=True)
)

# Tạo thư mục để lưu trữ chỉ mục
index_dir = "indexdir"
if not os.path.exists(index_dir):
    os.mkdir(index_dir)

# Tạo chỉ mục cho mặt hàng
ix_mat_hang = create_in(os.path.join(index_dir, "mat_hang"), schema_mat_hang)

# Thêm dữ liệu vào chỉ mục mặt hàng
def add_document(ix, doc):
    writer = ix.writer()
    writer.add_document(**doc)
    writer.commit()

# Cập nhật dữ liệu trong chỉ mục mặt hàng
def update_document(ix, doc_id, doc):
    writer = ix.writer()
    writer.update_document(id=doc_id, **doc)
    writer.commit()

# Xóa dữ liệu khỏi chỉ mục mặt hàng
def delete_document(ix, doc_id):
    writer = ix.writer()
    writer.delete_by_term('id', doc_id)
    writer.commit()

# Hàm thực hiện tìm kiếm
def search(query_str, field, index_name):
    ix = open_dir(os.path.join(index_dir, index_name))
    with ix.searcher() as searcher:
        query = QueryParser(field, ix.schema).parse(query_str)
        results = searcher.search(query)
        for result in results:
            print(result)

# Thêm tài liệu
add_document(ix_mat_hang, {"id": "1", "ten_mat_hang": "Sản phẩm A"})
add_document(ix_mat_hang, {"id": "2", "ten_mat_hang": "Sản phẩm B"})
add_document(ix_mat_hang, {"id": "3", "ten_mat_hang": "Sản phẩm C"})

# # Cập nhật tài liệu
# update_document(ix_mat_hang, "1", {"ten_mat_hang": "Sản phẩm A (Updated)"})

# # Xóa tài liệu
# delete_document(ix_mat_hang, "2")

# Tìm kiếm tài liệu
print("Tìm kiếm theo tên mặt hàng:")
search("Sản", "ten_mat_hang", "mat_hang")

In [13]:
import tkinter as tk
from tkinter import Menu, messagebox, Toplevel, Label, Entry, Button, Scrollbar, StringVar, font
from tkinter import ttk
# from PIL import Image, ImageTk
import sqlite3



# # Hàm tạo logo
# def create_logo(root):
#     # Tải hình ảnh logo
#     image = Image.open("path/to/logo.png")  # Đường dẫn đến file logo
#     photo = ImageTk.PhotoImage(image)

#     # Tạo Label để hiển thị logo
#     logo_label = tk.Label(root, image=photo)
#     logo_label.image = photo  # Giữ tham chiếu đến hình ảnh để tránh bị xóa bởi garbage collector
#     logo_label.grid(row=0, column=0, padx=10, pady=10, sticky="w")

# Hàm tạo menu
def create_menu(root):
    # Tạo menu chính
    menu_bar = Menu(root, font=custom_font)

    # Tạo menu File
    file_menu = Menu(menu_bar, tearoff=0, font=custom_font)
    file_menu.add_command(label="Mặt hàng", font=custom_font)
    file_menu.add_command(label="Nhập hàng", font=custom_font)
    file_menu.add_command(label="Bán hàng", font=custom_font)
    file_menu.add_command(label="Chi phí", font=custom_font)
    file_menu.add_command(label="Khách hàng", font=custom_font)
    file_menu.add_command(label="NCC", font=custom_font)
    file_menu.add_command(label="Báo cáo", font=custom_font)
    menu_bar.add_cascade(label="Điều hướng", menu=file_menu)

    # Gắn menu vào cửa sổ chính
    root.config(menu=menu_bar)

# Hàm thêm mặt hàng
def add_item():
    def save_item():
        # Lưu mặt hàng vào cơ sở dữ liệu
        conn = sqlite3.connect("sales_management.db")
        cursor = conn.cursor()
        cursor.execute('''INSERT INTO MatHang (id, ten_hang, don_vi, so_luong, gia_le, gia_si, ngay_tao, is_active)
                          VALUES (?, ?, ?, ?, ?, ?, ?, ?)''',
                       (id_var.get(), ten_hang_var.get(), don_vi_var.get(), int(so_luong_var.get()),
                        int(gia_le_var.get()), int(gia_si_var.get()), ngay_tao_var.get(), 1))
        conn.commit()
        conn.close()
        add_item_window.destroy()
        refresh_list()

    add_item_window = Toplevel(root)
    add_item_window.title("Thêm mặt hàng")

    Label(add_item_window, text="ID:").grid(row=0, column=0, padx=5, pady=5)
    id_var = StringVar()
    Entry(add_item_window, textvariable=id_var).grid(row=0, column=1, padx=5, pady=5)

    Label(add_item_window, text="Tên hàng:").grid(row=1, column=0, padx=5, pady=5)
    ten_hang_var = StringVar()
    Entry(add_item_window, textvariable=ten_hang_var).grid(row=1, column=1, padx=5, pady=5)

    Label(add_item_window, text="Đơn vị:").grid(row=2, column=0, padx=5, pady=5)
    don_vi_var = StringVar()
    Entry(add_item_window, textvariable=don_vi_var).grid(row=2, column=1, padx=5, pady=5)

    Label(add_item_window, text="Số lượng:").grid(row=3, column=0, padx=5, pady=5)
    so_luong_var = StringVar()
    Entry(add_item_window, textvariable=so_luong_var).grid(row=3, column=1, padx=5, pady=5)

    Label(add_item_window, text="Giá lẻ:").grid(row=4, column=0, padx=5, pady=5)
    gia_le_var = StringVar()
    Entry(add_item_window, textvariable=gia_le_var).grid(row=4, column=1, padx=5, pady=5)

    Label(add_item_window, text="Giá sỉ:").grid(row=5, column=0, padx=5, pady=5)
    gia_si_var = StringVar()
    Entry(add_item_window, textvariable=gia_si_var).grid(row=5, column=1, padx=5, pady=5)

    Label(add_item_window, text="Ngày tạo:").grid(row=6, column=0, padx=5, pady=5)
    ngay_tao_var = StringVar()
    Entry(add_item_window, textvariable=ngay_tao_var).grid(row=6, column=1, padx=5, pady=5)

    Button(add_item_window, text="Lưu", command=save_item).grid(row=7, column=0, columnspan=2, padx=5, pady=5)

# Hàm làm mới danh sách mặt hàng
def refresh_list():
    for item in tree.get_children():
        tree.delete(item)
    conn = sqlite3.connect("sales_management.db")
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM MatHang ORDER BY ten_hang")
    rows = cursor.fetchall()
    for row in rows:
        tree.insert("", tk.END, values=row)
    conn.close()

# Hàm xem chi tiết/sửa mặt hàng
def view_edit_item(item_id):
    def save_changes():
        # Lưu thay đổi vào cơ sở dữ liệu
        conn = sqlite3.connect("sales_management.db")
        cursor = conn.cursor()
        cursor.execute('''UPDATE MatHang
                          SET ten_hang = ?, don_vi = ?, so_luong = ?, gia_le = ?, gia_si = ?, is_active = ?
                          WHERE id = ?''',
                       (ten_hang_var.get(), don_vi_var.get(), int(so_luong_var.get()),
                        int(gia_le_var.get()), int(gia_si_var.get()), 1, item_id))
        conn.commit()
        conn.close()
        view_edit_window.destroy()
        refresh_list()

    view_edit_window = Toplevel(root)
    view_edit_window.title("Xem chi tiết/Sửa mặt hàng")

    conn = sqlite3.connect("sales_management.db")
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM MatHang WHERE id = ?", (item_id,))
    item = cursor.fetchone()
    conn.close()

    Label(view_edit_window, text="ID:").grid(row=0, column=0, padx=5, pady=5)
    Label(view_edit_window, text=item[0]).grid(row=0, column=1, padx=5, pady=5)

    Label(view_edit_window, text="Tên hàng:").grid(row=1, column=0, padx=5, pady=5)
    ten_hang_var = StringVar(value=item[1])
    Entry(view_edit_window, textvariable=ten_hang_var).grid(row=1, column=1, padx=5, pady=5)

    Label(view_edit_window, text="Đơn vị:").grid(row=2, column=0, padx=5, pady=5)
    don_vi_var = StringVar(value=item[2])
    Entry(view_edit_window, textvariable=don_vi_var).grid(row=2, column=1, padx=5, pady=5)

    Label(view_edit_window, text="Số lượng:").grid(row=3, column=0, padx=5, pady=5)
    so_luong_var = StringVar(value=item[3])
    Entry(view_edit_window, textvariable=so_luong_var).grid(row=3, column=1, padx=5, pady=5)

    Label(view_edit_window, text="Giá lẻ:").grid(row=4, column=0, padx=5, pady=5)
    gia_le_var = StringVar(value=item[4])
    Entry(view_edit_window, textvariable=gia_le_var).grid(row=4, column=1, padx=5, pady=5)

    Label(view_edit_window, text="Giá sỉ:").grid(row=5, column=0, padx=5, pady=5)
    gia_si_var = StringVar(value=item[5])
    Entry(view_edit_window, textvariable=gia_si_var).grid(row=5, column=1, padx=5, pady=5)

    Button(view_edit_window, text="Lưu", command=save_changes).grid(row=6, column=0, columnspan=2, padx=5, pady=5)

# Hàm xóa mặt hàng
def delete_item(item_id, item_name):
    def confirm_delete():
        conn = sqlite3.connect("sales_management.db")
        cursor = conn.cursor()
        cursor.execute("DELETE FROM MatHang WHERE id = ?", (item_id,))
        conn.commit()
        conn.close()
        delete_window.destroy()
        refresh_list()

    delete_window = Toplevel(root)
    delete_window.title("Xóa mặt hàng")

    Label(delete_window, text=f"Bạn có muốn xóa {item_name} này không?").grid(row=0, column=0, padx=5, pady=5)
    Button(delete_window, text="Có", command=confirm_delete).grid(row=1, column=0, padx=5, pady=5)
    Button(delete_window, text="Không", command=delete_window.destroy).grid(row=1, column=1, padx=5, pady=5)

# Tạo cửa sổ chính
root = tk.Tk()
root.geometry("800x600")
root.title("Sales Management - MatHang")
# Tạo phông chữ tùy chỉnh
custom_font = font.Font(family="Helvetica", size=12, weight="bold")
# Tạo logo và menu
# create_logo(root)
create_menu(root)

# Tiêu đề
Label(root, text="Quản lý mặt hàng", font=custom_font).grid(row=1, column=0, columnspan=4, padx=10, pady=10)

# Thông tin tổng mặt hàng và tổng số lượng hàng
total_items_label = Label(root, text="Tổng mặt hàng: 0", font=custom_font)
total_items_label.grid(row=2, column=2, padx=10, pady=10, sticky="e")

total_quantity_label = Label(root, text="Tổng số lượng hàng: 0", font=custom_font)
total_quantity_label.grid(row=2, column=3, padx=10, pady=10, sticky="e")

# Nút thêm mặt hàng
Button(root, text="Thêm mặt hàng", command=add_item, font=custom_font).grid(row=2, column=0, padx=10, pady=10, sticky="w")

# Bảng hiển thị danh sách mặt hàng
columns = ("id", "ten_hang", "don_vi", "so_luong", "gia_le", "gia_si", "ngay_tao", "is_active", "actions")
tree = ttk.Treeview(root, columns=columns, show="headings")
tree.heading("id", text="ID")
tree.heading("ten_hang", text="Tên hàng")
tree.heading("don_vi", text="Đơn vị")
tree.heading("so_luong", text="Số lượng")
tree.heading("gia_le", text="Giá lẻ")
tree.heading("gia_si", text="Giá sỉ")
tree.heading("ngay_tao", text="Ngày tạo")
tree.heading("is_active", text="Hoạt động")
tree.heading("actions", text="Hành động")

tree.column("actions", width=200)

tree.grid(row=3, column=0, columnspan=4, padx=10, pady=10)

# Scrollbar
scrollbar = Scrollbar(root, orient="vertical", command=tree.yview)
scrollbar.grid(row=3, column=4, sticky="ns")
tree.configure(yscrollcommand=scrollbar.set)

# Làm mới danh sách mặt hàng
refresh_list()

# Chạy vòng lặp chính của Tkinter
root.mainloop()

In [1]:
import tkinter as tk
from tkinter import Toplevel, Frame

frame = tk.Tk()
frame.title("Sales Management - MatHang")

# Tạo Label
label = tk.Label(frame, text="Hello, World!")
label.pack()

view_new_top_window = Toplevel(frame)
view_new_top_window.title("Thêm mới nhập hàng")
view_new_top_window.geometry("1080x720")
view_new_top_window.rowconfigure(0, weight=1)

sub_frame_left = Frame(view_new_top_window, bg='yellow')
sub_frame_left.grid(row=0, column=0, sticky="nsew")
sub_frame_right = Frame(view_new_top_window, bg='red')
sub_frame_right.grid(row=0, column=1, sticky="nsew")

sub_frame_right_top = Frame(sub_frame_right, bg='blue')
sub_frame_right_top.grid(row=0, column=0, sticky="nsew")
sub_frame_right_middle = Frame(sub_frame_right, bg='gray')
sub_frame_right_middle.grid(row=1, column=0, sticky="nsew")
sub_frame_right_bottom = Frame(sub_frame_right, bg='black')
sub_frame_right_bottom.grid(row=2, column=0, sticky="nsew")

view_new_top_window.columnconfigure(0, weight=1)
view_new_top_window.columnconfigure(1, weight=3)
# sub_frame_left.rowconfigure(0, weight=1)
# sub_frame_left.columnconfigure(0, weight=1)
sub_frame_right.rowconfigure(0, weight=1)
sub_frame_right.rowconfigure(1, weight=5)
sub_frame_right.rowconfigure(2, weight=1)
sub_frame_right.columnconfigure(0, weight=1)
# Chạy vòng lặp chính của Tkinter
frame.mainloop()


In [3]:
from datetime import datetime, timedelta
# import calendar

# class BaoCaoService:
#     def report_loi_nhuan(self, month: str, year: str):
#         # Xác định số ngày trong tháng
#         num_days = calendar.monthrange(int(year), int(month))[1]

#         # Lặp qua các ngày trong tháng
#         for day in range(1, num_days + 1):
#             date_str = f'{year}-{month}-{day:02d}'
#             date = datetime.strptime(date_str, '%Y-%m-%d')
#             # Thực hiện các thao tác cần thiết với từng ngày
#             print(f'Processing date: {date}')

#         # Trả về kết quả báo cáo (ví dụ: danh sách các ngày đã xử lý)
#         return [f'{year}-{month}-{day:02d}' for day in range(1, num_days + 1)]

# # Ví dụ sử dụng
# bao_cao_service = BaoCaoService()
# result = bao_cao_service.report_loi_nhuan('01', '2025')
# print(result)
# print(datetime.strptime('2025-01-01', '%Y-%m-%d').day)
print(datetime.strptime(datetime.now(), '%Y-%m-%d'))


TypeError: strptime() argument 1 must be str, not datetime.datetime

In [ ]:
import tkinter as tk
from tkinter import ttk, messagebox, filedialog

class NhapHangAddController:
    def __init__(self, frame: tk.Frame):
        self.frame = frame
        logging.info("NhapHangAdd Controller")
        self.nhap_hang_service = NhapHangService() 
        self.nhap_hang_list_var = []
        self.search_mat_hang_var = tk.StringVar()

        # Thêm nút để chọn thư mục lưu trữ
        self.select_folder_button = tk.Button(self.frame, text="Chọn thư mục lưu trữ", command=self.select_folder)
        self.select_folder_button.pack(pady=20)

        # Thêm nút để tạo và in hóa đơn
        self.print_button = tk.Button(self.frame, text="In hóa đơn", command=self.create_and_print_invoice)
        self.print_button.pack(pady=20)

    def select_folder(self):
        # Mở hộp thoại chọn thư mục
        folder_selected = filedialog.askdirectory()
        if folder_selected:
            messagebox.showinfo("Thư mục đã chọn", f"Thư mục đã chọn: {folder_selected}")
            self.selected_folder = folder_selected
        else:
            messagebox.showwarning("Chưa chọn thư mục", "Bạn chưa chọn thư mục nào.")

    def create_and_print_invoice(self):
        # Dữ liệu hóa đơn mẫu
        invoice_data = {
            "Mã hóa đơn": "HD001",
            "Ngày": "2023-12-01",
            "Khách hàng": "Nguyễn Văn A",
            "items": [
                {"name": "Sản phẩm A", "quantity": 2, "price": "100.000"},
                {"name": "Sản phẩm B", "quantity": 1, "price": "200.000"},
                {"name": "Sản phẩm C", "quantity": 3, "price": "150.000"},
            ],
            "total": "650.000"
        }

        # Tạo tài liệu PDF
        pdf_filename = f"{self.selected_folder}/invoice.pdf"
        self.create_invoice_pdf(pdf_filename, invoice_data)

        # In tài liệu PDF
        self.print_pdf(pdf_filename)

    def create_invoice_pdf(self, filename, invoice_data):
        c = canvas.Canvas(filename, pagesize=letter)
        width, height = letter

        # Tiêu đề hóa đơn
        c.setFont("Helvetica-Bold", 20)
        c.drawString(200, height - 50, "HÓA ĐƠN BÁN HÀNG")

        # Thông tin hóa đơn
        c.setFont("Helvetica", 12)
        y = height - 100
        for key, value in invoice_data.items():
            if key != "items":
                c.drawString(50, y, f"{key}: {value}")
                y -= 20

        # Thông tin sản phẩm
        c.drawString(50, y, "Sản phẩm:")
        y -= 20
        for item in invoice_data['items']:
            c.drawString(70, y, f"{item['name']} - Số lượng: {item['quantity']} - Giá: {item['price']}")
            y -= 20

        # Tổng cộng
        c.drawString(50, y, f"Tổng cộng: {invoice_data['total']}")

        c.save()

    def print_pdf(self, filename):
        if platform.system() == "Windows":
            os.startfile(filename, "print")
        elif platform.system() == "Darwin":
            os.system(f"lp {filename}")
        else:
            os.system(f"lpr {filename}")

if __name__ == "__main__":
    root = tk.Tk()
    root.geometry("800x600")
    app = NhapHangAddController(root)
    root.mainloop()

In [ ]:
# QL Chi phí
# crud: chi phí
# sort: Tên, ngày tạo, giá chi phí
# Tìm theo ngày/tháng/năm: nếu không chọn tháng năm thì mặc định là tháng năm hiện tại, không chọn ngày thì mặc định là lấy tất cả
# nút làm mới
# tổng chi phí

QL nhập hàng
rud: ud -> cập nhập/xóa vào chi phí, id nhập hàng là tên, ghi chú chi phí là Nhập hàng -> mặt hàng
sort: Tên, ngày nhập, giá nhập, số lượng, thanh toán
C -> thêm vào chi phí, id nhập hàng là tên, ghi chú chi phí là Nhập hàng -> mặt hàng
QL bán hàng
excel
in hóa đơn

# báo cáo: lợi nhuận, doanh thu, chi phí, bảng báo cáo theo từng ngày trong tháng: tổng chi phí + doanh thu + lợi nhuận, từng tháng trong năm
# báo cáo bán hàng: trong ngày bán được bao nhiêu, trong tháng
# báo cáo nhập hàng: trong ngày nhập được bao nhiêu, trong tháng
# sắp xếp theo tổng chi phí, doanh thu, lợi nhuận
# sắp xếp theo số lượng, thành tiền
# Giải phóng thuộc tính view_new_top_window bằng cách gán nó bằng None
